# Part1 Pre-Processing

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("part1").getOrCreate()

# Bussiness

In [3]:
path = 'yelp_academic_dataset_business.json'
business = spark.read.json(path)
business.count()

160585

In [4]:
business_col = ['business_id','name','city','state','stars','review_count','categories','latitude','longitude','is_open', 'postal_code']
business = business.select(business_col)
#business = business.withColumn('category', split(business['categories'],',')).withColumn('category',explode('category')).withColumn('category', trim('category'))


In [5]:
tmp = business.withColumn('category', explode(split(business['categories'],','))).withColumn('category', trim('category'))
tmp.groupBy('category').count().orderBy('count',ascending=False).show()
#selected_category = ['Restaurants','Food','Coffee & Tea','Sandwiches','Breakfast & Brunch']
regex_expr = r'\b(Restaurants|Food|Coffee|Tea|Sandwiches|Breakfast|Brunch)\b'
business = business.filter(business['categories'].rlike(regex_expr))


+--------------------+-----+
|            category|count|
+--------------------+-----+
|         Restaurants|50763|
|                Food|29469|
|            Shopping|26205|
|       Beauty & Spas|16574|
|       Home Services|16465|
|    Health & Medical|15102|
|      Local Services|12192|
|           Nightlife|11990|
|                Bars|10741|
|          Automotive|10119|
|Event Planning & ...| 9644|
|         Active Life| 9231|
|        Coffee & Tea| 7725|
|          Sandwiches| 7272|
|             Fashion| 6599|
|American (Traditi...| 6541|
|         Hair Salons| 5900|
|               Pizza| 5756|
|     Hotels & Travel| 5703|
|  Breakfast & Brunch| 5505|
+--------------------+-----+
only showing top 20 rows



In [6]:
business.count()

64092

In [7]:
business.groupBy('city').count().orderBy('count',ascending = False).show()
selected_city = ['Portland']
business = business.filter(col('city').isin(selected_city))


+---------------+-----+
|           city|count|
+---------------+-----+
|       Portland| 7212|
|      Vancouver| 6495|
|         Austin| 6373|
|        Atlanta| 5092|
|        Orlando| 4592|
|         Boston| 3536|
|       Columbus| 3164|
|       Richmond| 1127|
|      Cambridge|  957|
|        Burnaby|  919|
|        Boulder|  882|
|      Kissimmee|  846|
|      Beaverton|  735|
|        Decatur|  627|
|     Somerville|  510|
|North Vancouver|  500|
|    Winter Park|  495|
|         Quincy|  414|
|      Coquitlam|  383|
|         Surrey|  344|
+---------------+-----+
only showing top 20 rows



In [8]:
business = business[business['is_open']==1]
business.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in business.columns]).show()

+-----------+----+----+-----+-----+------------+----------+--------+---------+-------+-----------+
|business_id|name|city|state|stars|review_count|categories|latitude|longitude|is_open|postal_code|
+-----------+----+----+-----+-----+------------+----------+--------+---------+-------+-----------+
|          0|   0|   0|    0|    0|           0|         0|       0|        0|      0|          0|
+-----------+----+----+-----+-----+------------+----------+--------+---------+-------+-----------+



In [9]:
business.count()

4127

# Review

In [10]:
path = 'yelp_academic_dataset_review.json'
review = spark.read.json(path)
review.count()

8635403

In [11]:
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [12]:
review.show(10)
review = review.withColumn('year', substring('date',1,4).astype('int'))
review = review.withColumn('month', substring('date',6,2).astype('int'))
review = review.withColumn('day', substring('date',9,2).astype('int'))

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|buF9druCkbuXLX526...|   1|2014-10-11 03:34:02|    1|lWC-xP3rd6obsecCY...|  4.0|Apparently Prides...|     3|ak0TdVmGKo4pwqdJS...|
|RA4V8pr014UyUbDvI...|   0|2015-07-03 20:38:25|    0|8bFej1QE5LXp4O05q...|  4.0|This store is pre...|     1|YoVfDbnISlW0f7abN...|
|_sS2LBIGNT5NQb6PD...|   0|2013-05-28 20:38:06|    0|NDhkzczKjLshODbqD...|  5.0|I called WVM on t...|     0|eC5evKn1TWDyHCyQA...|
|0AzLzHfOJgL7ROwhd...|   1|2010-01-08 02:29:15|    1|T5fAqjjFooT4V0OeZ...|  2.0|I've stayed at ma...|     1|SFQ1jcnGguO0LYWnb...|
|8zehGz9jnxPqXtOc7...|   0|2011-07-28 18:05:01|    0|sjm_uUcQVxab_EeLC...|  4.0|The food i

In [13]:
review_col = ['review_id','business_id','user_id','year','month','day','stars','useful','funny','cool','text']
review = review.select(review_col)

In [14]:
review.show(10)

+--------------------+--------------------+--------------------+----+-----+---+-----+------+-----+----+--------------------+
|           review_id|         business_id|             user_id|year|month|day|stars|useful|funny|cool|                text|
+--------------------+--------------------+--------------------+----+-----+---+-----+------+-----+----+--------------------+
|lWC-xP3rd6obsecCY...|buF9druCkbuXLX526...|ak0TdVmGKo4pwqdJS...|2014|   10| 11|  4.0|     3|    1|   1|Apparently Prides...|
|8bFej1QE5LXp4O05q...|RA4V8pr014UyUbDvI...|YoVfDbnISlW0f7abN...|2015|    7|  3|  4.0|     1|    0|   0|This store is pre...|
|NDhkzczKjLshODbqD...|_sS2LBIGNT5NQb6PD...|eC5evKn1TWDyHCyQA...|2013|    5| 28|  5.0|     0|    0|   0|I called WVM on t...|
|T5fAqjjFooT4V0OeZ...|0AzLzHfOJgL7ROwhd...|SFQ1jcnGguO0LYWnb...|2010|    1|  8|  2.0|     1|    1|   1|I've stayed at ma...|
|sjm_uUcQVxab_EeLC...|8zehGz9jnxPqXtOc7...|0kA0PAJ8QFMeveQWH...|2011|    7| 28|  4.0|     0|    0|   0|The food is alway...|


In [15]:
review.count()

8635403

## Filter Reviews in Bussiness Dataset

In [16]:
business_id_set = set()
for i in list(business.select('business_id').collect()):
    business_id_set.add(i[0])

In [17]:
review.count()

8635403

In [19]:
review = review.filter(review['business_id'].isin(business_id_set))

In [20]:
review.count()

605597

## Remove Reviews Before 2010

In [21]:
review.groupBy('year').count().orderBy('year').show()

+----+-----+
|year|count|
+----+-----+
|2004|    1|
|2005|   72|
|2006|  607|
|2007| 3369|
|2008| 8183|
|2009|13023|
|2010|20037|
|2011|27092|
|2012|30442|
|2013|38252|
|2014|52003|
|2015|66122|
|2016|69181|
|2017|74100|
|2018|80903|
|2019|76977|
|2020|42588|
|2021| 2645|
+----+-----+



In [22]:
review = review.filter(review.year>2010)

In [23]:
review.count()

560305

# User

In [24]:
path = 'yelp_academic_dataset_user.json'
user = spark.read.json(path)

In [25]:
user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [26]:
user_col = ['user_id','name','review_count','yelping_since','friends','useful','funny','cool','fans','average_stars']
user = user.select(user_col)

In [27]:
user.show(10)

+--------------------+----------+------------+-------------------+--------------------+------+-----+-----+----+-------------+
|             user_id|      name|review_count|      yelping_since|             friends|useful|funny| cool|fans|average_stars|
+--------------------+----------+------------+-------------------+--------------------+------+-----+-----+----+-------------+
|q_QQ5kBBwlCcbL1s4...|      Jane|        1220|2005-03-14 20:26:35|xBDpTUbai0DXrvxCe...| 15038|10030|11291|1357|         3.85|
|dIIKEfOgo0KqUfGQv...|      Gabi|        2136|2007-08-10 19:01:51|XPzYf9_mwG2eXYP2B...| 21272|10289|18046|1025|         4.09|
|D6ErcUnFALnCQN4b1...|     Jason|         119|2007-02-07 15:47:53|GfB6sC4NJQvSI2ewb...|   188|  128|  130|  16|         3.76|
|JnPIjvC0cmooNDfsa...|       Kat|         987|2009-02-09 16:14:29|HQZPQhKMwRAyS6BCs...|  7234| 4722| 4035| 420|         3.77|
|37Hc8hr3cw0iHLoPz...| Christine|         495|2008-03-03 04:57:05|-Q88pZUcrfN0BLBDp...|  1577|  727| 1124|  47|       

In [28]:
# get active user
user = user[user.review_count>10]

In [29]:
user.count()

726250

## Save to CSV

In [53]:
# col = ['business_id','name','city','state','stars','review_count','categories','latitude','longitude','is_open', 'postal_code']
# city = ['Portland']
# category =  ['Restaurants','Food','Coffee & Tea','Sandwiches','Breakfast & Brunch']
# is_open = 1
business.coalesce(1).write.option("header",True).csv("part1_dataclean_business.csv")

In [54]:
# col = ['review_id','business_id','user_id','year','month','day','stars','useful','funny','cool','text']
# only include reviews which "business_id" in "bussiness.json"
review.coalesce(1).write.option("header",True).csv("part1_dataclean_review.csv")

In [55]:
# col = ['user_id','name','review_count','yelping_since','friends','useful','funny','cool','fans','average_stars']
user.coalesce(1).write.option("header",True).csv("part1_dataclean_user.csv")